In [17]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import json
import datetime

In [18]:
with open('mvdis.json', encoding="utf-8") as f:
    mvdis_data = json.load(f)

In [23]:
mvdis_url = "https://www.mvdis.gov.tw/m3-emv-plate/webpickno/queryPickNo"
captcha_url = "https://www.mvdis.gov.tw/m3-emv-plate/captchaImg.jpg"

h = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
     "Accept-Encoding": "gzip, deflate, br",
     "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6,zh-CN;q=0.5",
     "Cache-Control": "max-age=0",
     "Connection": "keep-alive",
     "Cookie": 'DWRSESSIONID=tNG0MdxettRHrCcz6yf*mYGBaao; _ga=GA1.3.1538469191.1659018978; _gid=GA1.3.204515142.1660153474; BSESSIONID1=CA803AF53E4DD6F31CF240E86960B19C.tsb12; JSESSIONID1=9BC6A27AC244DB24135BB0F45778CB69.tsp11; ADRUM_BTa="R:59|g:9af7fc30-d8db-400b-9a3c-159197f77717|n:customer1_b3bb7d45-77a0-4845-9079-72c1a6bc63f9"; SameSite=None; ADRUM_BT1="R:59|i:28292|e:381"; _gat=1; _gat_gtag_UA_81912319_9=1',
     "Host": "www.mvdis.gov.tw",
     "sec-ch-ua": '"Chromium";v="104", " Not A;Brand";v="99", "Google Chrome";v="104"',
     "sec-ch-ua-mobile": "?0",
     "sec-ch-ua-platform": '"Windows"',
     "Sec-Fetch-Dest": "document",
     "Sec-Fetch-Mode": "navigate",
     "Sec-Fetch-Site": "none",
     "Sec-Fetch-User": "?1",
     "Upgrade-Insecure-Requests": "1",
     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
     }

### Get csrf_token !

In [24]:
r = requests.get(
    mvdis_url,
    headers=h)

b = BeautifulSoup(r.text, features="html.parser")
b_csrf = b.find('input', {"name": "CSRFToken"})
csrf = b_csrf["value"]
print(csrf)

2feb631d-0f5a-4b06-b7c8-b1115d1ae71d


In [25]:
# payload constant
method = "qryPickNo"
selWindowNo = "01"
selCarType = "C"
selEnergyType = "E"
selPlateType = "g"
plateVer = "2"
queryType = "0"
queryNo = "*"

In [27]:
df = pd.DataFrame(columns=["plate", "selDeptName", "selStationName", "location"])
plates = []
# for i in range(len(mvdis_data)):
for i in range(3):
    craw = True
    while craw:
        craw = False
        r_captcha = requests.get(
        captcha_url,
        headers=h)
        img_name = "./captcha/"+("000000"+str(len(os.listdir("./captcha"))))[-6:]
        with open(img_name+".jpg", 'wb') as f:
            f.write(r_captcha.content)

        cap = input(f"cap{i}: ").upper()
        os.rename(img_name+".jpg", img_name+"_"+cap+".jpg")
        payload = {
            "method" : "qryPickNo",
            "selDeptCode" : mvdis_data[i]["selDeptCode"],
            "selStationCode" : mvdis_data[i]["selStationCode"],
            "selWindowNo" : "01",
            "location" : mvdis_data[i]["location"],
            "selCarType" : "C",
            "selEnergyType" : "E",
            "selPlateType" : "g",
            "plateVer" : "2",
            "validateStr" : cap,
            "queryType" : "0",
            "queryNo" : "*",
            "CSRFToken" : csrf
        }
        r_p = requests.post(
            mvdis_url,
            headers=h,
            data=payload
        )
        b_p = BeautifulSoup(r_p.text, features="html.parser")
        # nums = b_p.find_all('div', {"class": "number_cell"})
        
        if b_p.find('td', {"id":"headerMessage"}).text == "驗證數字輸入錯誤":
            craw = True
            continue
        
        
        nums = b_p.find_all('a', {"class": "number"})
        for num in nums:
            plates.append([
                num.text, 
                mvdis_data[i]["selDeptName"], 
                mvdis_data[i]["selStationName"], 
                mvdis_data[i]["location"]
            ])
            df_tmp = pd.DataFrame([[
                        num.text, mvdis_data[i]["selDeptName"], 
                        mvdis_data[i]["selStationName"], 
                        mvdis_data[i]["location"]]],
                        columns=["plate", "selDeptName", "selStationName", "location"])
            df = pd.concat([df, df_tmp], ignore_index=True)
        
        try:
            total_count = int((b_p.find("div", {"class": "align_c note"})).text.split()[1])
        except:
            continue
        
        if total_count>65:
            page_params={
                "d-3611679-p": 2,
                "method": "queryPickNoView"
            }
            r_page2 = requests.get(
                mvdis_url,
                headers=h,
                params=page_params
            )
            b_page2 = BeautifulSoup(r_page2.text, features="html.parser")
            nums_page2 = b_page2.find_all('a', {"class": "number"})
            for num in nums_page2:
                plates.append([
                    num.text, 
                    mvdis_data[i]["selDeptName"], 
                    mvdis_data[i]["selStationName"], 
                    mvdis_data[i]["location"]
                ])
                df_tmp = pd.DataFrame([[
                        num.text, mvdis_data[i]["selDeptName"], 
                        mvdis_data[i]["selStationName"], 
                        mvdis_data[i]["location"]]],
                        columns=["plate", "selDeptName", "selStationName", "location"])
                df = pd.concat([df, df_tmp], ignore_index=True)

cap0: 8fvn
cap1: 86le
cap1: lhgm
cap2: hxny


In [28]:
df

,plate,selDeptName,selStationName,location
0,EAD-5021,台北市,台北市區監理所,臺北市八德路4段21號地下室
1,EAD-5023,台北市,台北市區監理所,臺北市八德路4段21號地下室
2,EAD-5027,台北市,台北市區監理所,臺北市八德路4段21號地下室
3,EAD-5031,台北市,台北市區監理所,臺北市八德路4段21號地下室
4,EAD-5032,台北市,台北市區監理所,臺北市八德路4段21號地下室
...,...,...,...,...
151,EAD-0956,台北市,基隆監理站,基隆市七堵區實踐路296號一樓
152,EAD-0961,台北市,基隆監理站,基隆市七堵區實踐路296號一樓
153,EAD-0967,台北市,基隆監理站,基隆市七堵區實踐路296號一樓
154,EAD-0971,台北市,基隆監理站,基隆市七堵區實踐路296號一樓


In [29]:
df.to_csv(f"mvdis_{datetime.date.today()}.csv", encoding="utf-8", index=False)

In [22]:
n = f"mvdis_{datetime.date.today()}.csv"
print(n)

mvdis_2022-08-17.csv


In [32]:
df.sort_values(by="plate")

,plate,selDeptName,selStationName,location
120,EAC-0793,台北市,基隆監理站,基隆市七堵區實踐路296號一樓
100,EAD-0352,台北市,士林監理站,台北市士林區承德路5段80號
101,EAD-0361,台北市,士林監理站,台北市士林區承德路5段80號
102,EAD-0362,台北市,士林監理站,台北市士林區承德路5段80號
103,EAD-0363,台北市,士林監理站,台北市士林區承德路5段80號
...,...,...,...,...
95,EAD-5292,台北市,台北市區監理所,臺北市八德路4段21號地下室
96,EAD-5293,台北市,台北市區監理所,臺北市八德路4段21號地下室
97,EAD-5295,台北市,台北市區監理所,臺北市八德路4段21號地下室
98,EAD-5296,台北市,台北市區監理所,臺北市八德路4段21號地下室


In [33]:
dff = pd.read_csv("./plates/mvdis_2022-08-17.csv")
dff_sorted = dff.sort_values(by="plate")
dff_sorted

,plate,selDeptName,selStationName,location
249,EAA-2571,台北區,宜蘭監理站,宜蘭縣五結鄉中正路二段9號一樓
250,EAA-2573,台北區,宜蘭監理站,宜蘭縣五結鄉中正路二段9號一樓
251,EAA-2587,台北區,宜蘭監理站,宜蘭縣五結鄉中正路二段9號一樓
252,EAA-2592,台北區,宜蘭監理站,宜蘭縣五結鄉中正路二段9號一樓
253,EAA-2593,台北區,宜蘭監理站,宜蘭縣五結鄉中正路二段9號一樓
...,...,...,...,...
204,EAE-7622,高雄市,高雄市區監理所,高雄市楠梓區德民路71號
205,EAE-7623,高雄市,高雄市區監理所,高雄市楠梓區德民路71號
206,EAE-7625,高雄市,高雄市區監理所,高雄市楠梓區德民路71號
207,EAE-7626,高雄市,高雄市區監理所,高雄市楠梓區德民路71號


In [34]:
dff_sorted.to_csv("./plates/mvdis_2022-08-17_sorted.csv")